<a href="https://colab.research.google.com/github/KevinBarranco24/Practica2_RW2/blob/main/Practica_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 2
##Random Walks II

###Kevin Kristop Morales Barranco - 214603492



##MODULES

In [1]:
import math
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from scipy.stats import exponweib

import pandas as pd
from scipy.stats import levy_stable

#**Actividad 1: Lévy distribution - N Curvas (3 pts)**
 * Generar al menos las 4 curvas de distribución tipo Lévy stable que se observan en la imagen y graficarlas en una sola figura con plotly.
 * Cada una de las curvas corresponde a un valor distinto de alpha.
 * Considere los siguientes parametros para las curvas a generar: m = 3.0, beta = 0.

In [3]:
miu = 3.0
Levy_exponent = [0.7, 0.5, 0.1, 0.6]
beta = 0
Levy_pdf = list()

aux_domain = np.linspace(miu-5, miu+5, 400)

for i in range(len(Levy_exponent)):
  Levy_pdf.append(np.array([levy_stable.pdf(i, alpha=Levy_exponent[i], beta=beta, loc=miu) for j in aux_domain]))